In [ ]:
import random
import altair as alt
import pandas as pd
from sklearn.compose import make_column_transformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics.pairwise import euclidean_distances
from sklearn import set_config
import matplotlib.pyplot as plt
import seaborn as sns



#Simplifyinf work with large datasets in ALtair
alt.data_transformers.disable_max_rows()

#Outputs Dataframes instead of Arrays
set_config(transform_output="pandas")

**Reading Data from Web**

In [500]:
url= "https://drive.google.com/uc?export=download&id=1_MECmUXZuuILYeEOfonSGqodW6qVdhsS"

In [501]:
players= pd.read_csv(url)


**Data Summary**


In [502]:
columns=players.columns.to_list()
columns

['Unnamed: 0',
 'Age',
 'Country',
 'Plays',
 'Wikipedia',
 'Current Rank',
 'Best Rank',
 'Name',
 'Backhand',
 'Prize Money',
 'Height',
 'Favorite Surface',
 'Turned Pro',
 'Seasons',
 'Active',
 'Current Elo Rank',
 'Best Elo Rank',
 'Peak Elo Rating',
 'Last Appearance',
 'Titles',
 'GOAT Rank',
 'Best Season',
 'Retired',
 'Masters',
 'Birthplace',
 'Residence',
 'Weight',
 'Coach',
 'Facebook',
 'Twitter',
 'Nicknames',
 'Grand Slams',
 'Davis Cups',
 'Web Site',
 'Team Cups',
 'Olympics',
 'Weeks at No. 1',
 'Tour Finals']

In [503]:
players

,Unnamed: 0,Age,Country,Plays,Wikipedia,Current Rank,Best Rank,Name,Backhand,Prize Money,...,Facebook,Twitter,Nicknames,Grand Slams,Davis Cups,Web Site,Team Cups,Olympics,Weeks at No. 1,Tour Finals
0,0,26 (25-04-1993),Brazil,Right-handed,Wikipedia,378 (97),363 (04-11-2019),Oscar Jose Gutierrez,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,18 (22-12-2001),United Kingdom,Left-handed,Wikipedia,326 (119),316 (14-10-2019),Jack Draper,Two-handed,"$59,040",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,32 (03-11-1987),Slovakia,Right-handed,Wikipedia,178 (280),44 (14-01-2013),Lukas Lacko,Two-handed,"US$3,261,567",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,21 (29-05-1998),"Korea, Republic of",Right-handed,Wikipedia,236 (199),130 (10-04-2017),Duck Hee Lee,Two-handed,"$374,093",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,27 (21-10-1992),Australia,Right-handed,Wikipedia,183 (273),17 (11-01-2016),Bernard Tomic,Two-handed,"US$6,091,971",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,20 (13-04-1999),France,Right-handed,Wikipedia,382 (95),380 (11-11-2019),Dan Added,Two-handed,"$57,943",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
496,496,26 (03-09-1993),Austria,Right-handed,Wikipedia,5 (5890),4 (06-11-2017),Dominic Thiem,One-handed,"$22,132,368 15th all-time leader in earnings",...,1.Dominic.Thiem,@ThiemDomi,Dominator,NaN,NaN,dominicthiem.tennis,NaN,NaN,NaN,NaN
497,497,23 (14-03-1996),Netherlands,Left-handed,Wikipedia,495 (60),342 (05-08-2019),Gijs Brouwer,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
498,498,24 (17-05-1995),Ukraine,NaN,Wikipedia,419 (81),419 (20-01-2020),Vladyslav Orlov,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [504]:
missing_count=players.isnull().sum()
missing_percentage = (missing_count / len(players)) * 100
column_type= players.dtypes

# Create a DataFrame to display the missing values count and percentage
missing_data = pd.DataFrame({'Missing Count': missing_count, 'Missing Percentage': missing_percentage, "Data Type" : column_type})
missing_data = missing_data.sort_values(by='Missing Count', ascending=True)

print(missing_data)

                  Missing Count  Missing Percentage Data Type
Unnamed: 0                    0                 0.0     int64
Name                          0                 0.0    object
Age                           1                 0.2    object
Country                       1                 0.2    object
Wikipedia                     1                 0.2    object
Best Rank                     1                 0.2    object
Current Rank                  5                 1.0    object
Plays                        47                 9.4    object
Prize Money                  81                16.2    object
Backhand                     92                18.4    object
Seasons                     126                25.2   float64
Last Appearance             158                31.6    object
Active                      218                43.6    object
Turned Pro                  254                50.8   float64
Favorite Surface            259                51.8    object
Best Elo

Given substantial amounts of missing values in some columns, these will be dropped later

Describing count, mean, min, max, 25%, 50%, 75% for each variable

In [527]:
round(players.describe(),2)

,Unnamed: 0,Prize Money,Turned Pro,Seasons,Titles,Best Season,Retired,Masters,Grand Slams,Davis Cups,...,Country_Uruguay,Country_Uzbekistan,Country_Zimbabwe,Country_encoded,Plays_Left-handed,Plays_Right-handed,Plays_encoded,Backhand_One-handed,Backhand_Two-handed,Backhand_encoded
count,344.00,3.440000e+02,221.00,344.00,75.00,80.00,78.00,5.0,1.0,21.00,...,344.00,344.00,344.00,344.00,344.00,344.00,344.00,344.00,344.00,344.00
mean,236.28,2.170744e+06,2009.63,6.03,4.08,2016.10,2016.73,8.0,19.0,1.33,...,0.00,0.01,0.00,31.44,0.12,0.87,0.88,0.08,0.89,0.94
std,140.10,6.910640e+06,4.56,4.49,9.66,3.61,1.93,15.1,NaN,0.97,...,0.05,0.09,0.05,18.55,0.33,0.34,0.34,0.28,0.31,0.33
min,2.00,2.893000e+03,1998.00,1.00,1.00,2006.00,2008.00,1.0,19.0,1.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,120.75,2.034672e+05,2006.00,2.00,1.00,2015.00,2016.00,1.0,19.0,1.00,...,0.00,0.00,0.00,16.00,0.00,1.00,1.00,0.00,1.00,1.00
50%,230.50,5.930600e+05,2010.00,5.00,2.00,2018.00,2017.00,1.0,19.0,1.00,...,0.00,0.00,0.00,31.00,0.00,1.00,1.00,0.00,1.00,1.00
75%,349.50,2.068637e+06,2013.00,9.00,4.00,2019.00,2018.00,2.0,19.0,1.00,...,0.00,0.00,0.00,48.25,0.00,1.00,1.00,0.00,1.00,1.00
max,499.00,1.196016e+08,2019.00,21.00,84.00,2020.00,2019.00,35.0,19.0,5.00,...,1.00,1.00,1.00,61.00,1.00,1.00,2.00,1.00,1.00,2.00


**Preliminary Data Cleaning**

In [506]:
#extracting actuale Age from the column Age since it had format "Age: Date of Birth"df=df.dropna(inplace=True)
df= pd.DataFrame(players)

df["Clean_Age"]= pd.to_numeric(df["Age"].str.split().str[0])
df["Clean_Best_Rank"]= pd.to_numeric(df["Best Rank"].str.split().str[0])
df["Clean_Height"]= pd.to_numeric(df["Height"].str.split().str[0])
df["Clean_Current_Rank"]= pd.to_numeric(df["Current Rank"].str.split().str[0])

# also removing '$' and 'US$' from the data in the column
df['Prize Money'] = df['Prize Money'].str.replace(",", "", regex= False).str.replace('US$', '', regex=False).str.replace('$', '',  regex=False)
# Convert the values to float
df['Prize Money'] = pd.to_numeric(df['Prize Money'], errors='coerce')


In [507]:
#trnasforming categorical values to numerical
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

# Turning the "Country" column values into numerical
country_encoded = pd.get_dummies(df['Country'], prefix='Country')
country_encoded['Country_encoded'] = label_encoder.fit_transform(df['Country'])
country = df["Country"]
             
# Turning the "Plays" column values into numerical
plays_encoded = pd.get_dummies(df['Plays'], prefix='Plays')
plays_encoded['Plays_encoded'] = label_encoder.fit_transform(df['Plays'])

# Turning the "Backhand" column values into numerical
backhand_encoded = pd.get_dummies(df['Backhand'], prefix='Backhand')
backhand_encoded['Backhand_encoded'] = label_encoder.fit_transform(df['Backhand'])


# Concatenating the results
players = pd.concat([df, country_encoded, plays_encoded, backhand_encoded], axis=1)
players

,Unnamed: 0,Age,Country,Plays,Wikipedia,Current Rank,Best Rank,Name,Backhand,Prize Money,...,Country_Uruguay,Country_Uzbekistan,Country_Zimbabwe,Country_encoded,Plays_Left-handed,Plays_Right-handed,Plays_encoded,Backhand_One-handed,Backhand_Two-handed,Backhand_encoded
0,0,26 (25-04-1993),Brazil,Right-handed,Wikipedia,378 (97),363 (04-11-2019),Oscar Jose Gutierrez,NaN,NaN,...,0,0,0,8,0,1,1,0,0,2
1,1,18 (22-12-2001),United Kingdom,Left-handed,Wikipedia,326 (119),316 (14-10-2019),Jack Draper,Two-handed,59040.0,...,0,0,0,57,1,0,0,0,1,1
2,2,32 (03-11-1987),Slovakia,Right-handed,Wikipedia,178 (280),44 (14-01-2013),Lukas Lacko,Two-handed,3261567.0,...,0,0,0,47,0,1,1,0,1,1
3,3,21 (29-05-1998),"Korea, Republic of",Right-handed,Wikipedia,236 (199),130 (10-04-2017),Duck Hee Lee,Two-handed,374093.0,...,0,0,0,34,0,1,1,0,1,1
4,4,27 (21-10-1992),Australia,Right-handed,Wikipedia,183 (273),17 (11-01-2016),Bernard Tomic,Two-handed,6091971.0,...,0,0,0,1,0,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,20 (13-04-1999),France,Right-handed,Wikipedia,382 (95),380 (11-11-2019),Dan Added,Two-handed,57943.0,...,0,0,0,24,0,1,1,0,1,1
496,496,26 (03-09-1993),Austria,Right-handed,Wikipedia,5 (5890),4 (06-11-2017),Dominic Thiem,One-handed,NaN,...,0,0,0,2,0,1,1,1,0,0
497,497,23 (14-03-1996),Netherlands,Left-handed,Wikipedia,495 (60),342 (05-08-2019),Gijs Brouwer,NaN,NaN,...,0,0,0,39,1,0,0,0,0,2
498,498,24 (17-05-1995),Ukraine,NaN,Wikipedia,419 (81),419 (20-01-2020),Vladyslav Orlov,NaN,NaN,...,0,0,0,56,0,0,2,0,0,2


In [508]:
#dropping null values in columns:

players=players.dropna(subset=["Clean_Age", "Clean_Current_Rank", "Clean_Best_Rank", "Seasons", "Country_encoded", "Plays_encoded", "Backhand_encoded", "Prize Money", ])


In the project, we will use columns with low amount of missing values. Since some of them are of type object, work will be further done to convert them to types possible to build  prediction model. 

**Data Visualizations**

In [509]:

#the number of players in each age 
df["Claen_Age"]= df["Clean_Age"].astype(str)

age_counts = df['Clean_Age'].value_counts().sort_index().reset_index()
age_counts.columns = ['Clean_Age', 'Player Count']

plot1 = alt.Chart(age_counts).mark_bar().encode(
    x=alt.X("Clean_Age", title="Age"),
    y=alt.Y("Player Count", title="Count")
).properties(
    title='Age of the players'
)

In [510]:

bins = [0, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500]  # Define custom bin intervals

df['Rank Group'] = pd.cut(df['Clean_Best_Rank'], bins=bins)

# Count the number of players in each rank group
rank_counts = df['Rank Group'].value_counts().sort_index().reset_index()
rank_counts.columns = ['Rank Group', 'Player Count']
rank_counts['Rank Group'] = rank_counts['Rank Group'].astype(str)  # Convert Interval to string

plot2 = alt.Chart(rank_counts).mark_bar().encode(
    x='Rank Group:O',  # O: Ordinal (categorical) axis
    y='Player Count:Q',  # Q: Quantitative axis (numerical)
    tooltip=['Rank Group', 'Player Count']  # Hover tooltip with data
).properties(
    title='Number of Players in their Best Rank Groups'
).configure_axisX(
    labelAngle=45
).configure_view(
    width=500)


In [511]:
df['Current Rank Group'] = pd.cut(df['Clean_Current_Rank'], bins=bins)
df["Current Rank Group"]= df["Current Rank Group"].astype(str)

# Count the number of players in each rank group
rank_counts = df['Current Rank Group'].value_counts().sort_index().reset_index()
rank_counts.columns = ['Current Rank Group', 'Player Count']


plot3 = alt.Chart(rank_counts).mark_bar().encode(
    x='Current Rank Group:O',  # O: Ordinal (categorical) axis
    y='Player Count:Q',  # Q: Quantitative axis (numerical)
    tooltip=['Current Rank Group', 'Player Count']  # Hover tooltip with data
).properties(
    title='Number of Players in their Current Rank Groups'
).configure_axisX(
    labelAngle=45
).configure_view(
    width=500)


In [512]:
plot1

alt.Chart(...)

In [513]:
plot2

alt.Chart(...)

In [514]:
plot3

alt.Chart(...)

In [515]:
players["Country"]

2                Slovakia
3      Korea, Republic of
4               Australia
5                  Poland
6           United States
              ...        
491              Bulgaria
492               Ecuador
493                 India
494    Russian Federation
499               Tunisia
Name: Country, Length: 344, dtype: object

In [516]:

country_counts= players["Country"].value_counts()

top_countries = country_counts[country_counts > 5]

filtered_df = df.copy()
filtered_df['Country'] = filtered_df['Country'].apply(lambda x: x if x in top_countries else 'others')

final_counts = filtered_df['Country'].value_counts()

plot_data = pd.DataFrame({
    'Country': final_counts.index,
    'Players': final_counts.values
})

chart = alt.Chart(plot_data).mark_bar().encode(
    x='Country',
    y='Players',
    tooltip=['Country', 'Players']
).properties(
    width=600,
    title="Top countries in number of players"
).configure_axisX(
    labelAngle=45
)

chart

alt.Chart(...)

**Buidilng a model**

In [517]:
import altair as alt
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import set_config


In [518]:
#Now, let's split the set into trainig and testing. train size is 75%

players_train, players_test = train_test_split(
    players, train_size=0.75
)

In [519]:
from sklearn.neighbors import KNeighborsRegressor

players_preprocessor = make_column_transformer((StandardScaler(), ["Clean_Age", "Clean_Current_Rank", "Seasons", "Country_encoded", "Plays_encoded", "Backhand_encoded", "Prize Money"]))
players_pipeline = make_pipeline(players_preprocessor, KNeighborsRegressor())

players_grid = {
    "kneighborsregressor__n_neighbors": range(1, 201, 3),
}
players_gridsearch = GridSearchCV(
    estimator=players_pipeline,
    param_grid=players_grid,
    cv=5,
    scoring="neg_root_mean_squared_error",
)

In [520]:
print(players_train.columns)


Index(['Unnamed: 0', 'Age', 'Country', 'Plays', 'Wikipedia', 'Current Rank',
       'Best Rank', 'Name', 'Backhand', 'Prize Money',
       ...
       'Country_Uruguay', 'Country_Uzbekistan', 'Country_Zimbabwe',
       'Country_encoded', 'Plays_Left-handed', 'Plays_Right-handed',
       'Plays_encoded', 'Backhand_One-handed', 'Backhand_Two-handed',
       'Backhand_encoded'],
      dtype='object', length=111)


In [521]:
# fit the GridSearchCV object
players_gridsearch.fit(
    players_train[["Clean_Age", "Clean_Current_Rank", "Seasons", "Country_encoded", "Plays_encoded", "Backhand_encoded", "Prize Money"]],  # A single-column data frames
    players_train["Clean_Best_Rank"]  # A series
)



GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('standardscaler',
                                                                         StandardScaler(),
                                                                         ['Clean_Age',
                                                                          'Clean_Current_Rank',
                                                                          'Seasons',
                                                                          'Country_encoded',
                                                                          'Plays_encoded',
                                                                          'Backhand_encoded',
                                                                          'Prize '
                                                                          'Money'])])),
                                       ('kneighborsregressor',
                                        KNeighborsRegressor())]),
             param_grid={'kneighborsregressor__n_neighbors': range(1, 201, 3)},
             scoring='neg_root_mean_squared_error')

In [522]:
# Retrieve the CV scores
players_results = pd.DataFrame(players_gridsearch.cv_results_)
players_results["sem_test_score"] = players_results["std_test_score"] / 5**(1/2)
players_results = (
    players_results[[
        "param_kneighborsregressor__n_neighbors",
        "mean_test_score",
        "sem_test_score"
    ]]
    .rename(columns={"param_kneighborsregressor__n_neighbors": "n_neighbors"})
)


In [523]:
players_results["mean_test_score"]=-players_results["mean_test_score"]
players_results

,n_neighbors,mean_test_score,sem_test_score
0,1,67.616822,1.953477
1,4,61.055366,1.199169
2,7,61.368119,1.697779
3,10,62.162195,1.424378
4,13,62.266510,1.609773
...,...,...,...
62,187,99.654755,4.560964
63,190,100.479603,4.636997
64,193,101.316947,4.664447
65,196,102.143989,4.701371


In [524]:
players_gridsearch.best_params_

{'kneighborsregressor__n_neighbors': 4}

so knn=19 is the best number of neighbours to build the most accuarate model.

In [525]:
plot_neighbours= alt.Chart(players_results).mark_line().encode(
    x="n_neighbors",
    y="mean_test_score"
)
plot_neighbours

alt.Chart(...)

In [526]:
from sklearn.metrics import mean_squared_error

players_test["predicted"] = players_gridsearch.predict(players_test)

RMSPE = mean_squared_error(
    y_true=players_test["Clean_Best_Rank"],
    y_pred=players_test["predicted"]
)**(1/2)
RMSPE

48.805424665135185

Citation: Data Derived from HTML source: https://drive.google.com/uc?export=download&id=1_MECmUXZuuILYeEOfonSGqodW6qVdhsS